In [1]:
import sentence_transformers  # dependency for sentence_feature_extractor.py
# check that we can use sentence_transformers
print(sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2').encode(['hello world']).shape)
# import openai  # dependency for chatgpt.py
import evadb
cursor = evadb.connect(evadb_dir=("./eva_db")).cursor()


/home/oscar/.cache/pypoetry/virtualenvs/videos-ZS71cQZv-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/oscar/.cache/pypoetry/virtualenvs/videos-ZS71cQZv-py3.10/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


(1, 384)


In [2]:
# Register the function as embed text
cursor.query("""
DROP FUNCTION IF EXISTS EmbedText
""").df()
cursor.query("""
CREATE FUNCTION EmbedText IMPL './sentence_feature_extractor.py'
""").df()
cursor.query("""
SELECT EmbedText('hello world')
""").df()

,features
0,"[[-0.03447732, 0.031023178, 0.0067349477, 0.02..."


In [3]:
# add the data to evadb from games.db
# we will just copy the data from games.db to evadb
# create the games table
cursor.query("""
DROP TABLE IF EXISTS videos
""").df()
cursor.query("""
CREATE TABLE videos (
            start_time TEXT(0),
            src_url TEXT(0),
            transcript TEXT(0),
             video_id TEXT(0)
            )"""
).df()
cursor.query("""
SELECT * FROM videos
""").df()


""


In [4]:
from youtube_transcript_api import YouTubeTranscriptApi
import re

def get_video_id(youtube_url):
    """Extracts the video ID from the YouTube URL."""
    return re.search(r"(?<=v=)[^&#]+", youtube_url).group()

def get_transcript(video_id):
    """Fetches the transcript for a given video ID."""
    try:
        return YouTubeTranscriptApi.get_transcript(video_id)
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None

def create_chunks(transcript, chunk_length=20):
    """Splits the transcript into chunks of specified length (in seconds)."""
    chunks = []
    current_chunk = ""
    chunk_start_time = 0

    for entry in transcript:
        if entry['start'] < chunk_start_time + chunk_length:
            current_chunk += entry['text'] + " "
        else:
            chunks.append((chunk_start_time, current_chunk.strip()))
            chunk_start_time = entry['start']
            current_chunk = entry['text'] + " "

    # Add the last chunk
    if current_chunk:
        chunks.append((chunk_start_time, current_chunk.strip()))

    return chunks

def main(youtube_url):
    video_id = get_video_id(youtube_url)
    transcript = get_transcript(video_id)
    if transcript:
        chunks = create_chunks(transcript)
        for start_time, chunk in chunks:
            print(f"Start Time: {start_time}s, Transcript: {chunk}\n")


In [5]:
# fetch
youtube_url = "https://www.youtube.com/watch?v=L_Guz73e6fw"
video_id = get_video_id(youtube_url)
transcript = get_transcript(video_id)
chunks = create_chunks(transcript)

for start_time, chunk in chunks:
    print(start_time,'&&', chunk)
    print(youtube_url)
    bad_chars = '{}-;:"\''
    bad_chars_translation = str.maketrans(dict.fromkeys(bad_chars, ''))
    chunk = chunk.translate(bad_chars_translation)
    cursor.query(f"""
    INSERT INTO videos (start_time, src_url, transcript, video_id) VALUES ('{start_time}', '{youtube_url.replace(':','')}', '{chunk}', '{video_id}')
    """).df()

0 && - We have been a misunderstood and badly mocked org for a long time. Like, when we started, we,
like, announced the org at the end of 2015 and said
we were gonna work on AGI. Like, people thought
we were batshit insane. - Yeah. - You know, like, I remember at the time an eminent AI scientist at a
large industrial AI lab was,
https://www.youtube.com/watch?v=L_Guz73e6fw
24.84 && like, DM'ing individual reporters being, like, you know, these
people aren't very good and it's ridiculous to talk about AGI and I can't believe you're
giving them time of day. And it's, like, that was the level of, like, pettiness and rancor in the field at a new group of people saying, we're gonna try to build AGI. - So, OpenAI and DeepMind was
a small collection of folks who were brave enough to talk about AGI
https://www.youtube.com/watch?v=L_Guz73e6fw
48.677 && in the face of mockery. - We don't get mocked as much now. - We don't get mocked as much now. The following is a
conversation with Sam Altman, C

In [6]:
print(type(video_id))
print(video_id)

<class 'str'>
L_Guz73e6fw


In [2]:
# check that the data is in evadb
cursor.query("""
SELECT * FROM videos
""").df()

,_row_id,start_time,src_url,transcript,video_id
0,1,0,https//www.youtube.com/watch?v=L_Guz73e6fw,We have been a misunderstood and badly mocked...,L_Guz73e6fw
1,2,24.84,https//www.youtube.com/watch?v=L_Guz73e6fw,"like, DMing individual reporters being, like, ...",L_Guz73e6fw
2,3,48.677,https//www.youtube.com/watch?v=L_Guz73e6fw,in the face of mockery. We dont get mocked as...,L_Guz73e6fw
3,4,68.88,https//www.youtube.com/watch?v=L_Guz73e6fw,which both individually and together constitut...,L_Guz73e6fw
4,5,89.61,https//www.youtube.com/watch?v=L_Guz73e6fw,We stand on the precipice\nof fundamental soci...,L_Guz73e6fw
...,...,...,...,...,...
394,395,8533.83,https//www.youtube.com/watch?v=L_Guz73e6fw,"number one for me for\nAGI, how many others? A...",L_Guz73e6fw
395,396,8555.61,https//www.youtube.com/watch?v=L_Guz73e6fw,I think the challenges are tough. I understand...,L_Guz73e6fw
396,397,8578.89,https//www.youtube.com/watch?v=L_Guz73e6fw,but I think that also means we will have new t...,L_Guz73e6fw
397,398,8600.76,https//www.youtube.com/watch?v=L_Guz73e6fw,"And now, let me leave you with some words from...",L_Guz73e6fw


In [3]:
# create a vector index using faiss
cursor.query("""
DROP INDEX IF EXISTS videos_vector_idx
""").df()
cursor.query("""
CREATE INDEX videos_vector_idx
ON videos (EmbedText(videos.transcript))
USING FAISS
""").df()

/home/oscar/.cache/pypoetry/virtualenvs/videos-ZS71cQZv-py3.10/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[idx] for idx in length_sorted_idx]


,0
0,Index videos_vector_idx successfully added to ...


In [3]:
test_query = "Why does Sam Altman feel defensive?"
cursor.query(f"""
SELECT * FROM videos ORDER BY Similarity(EmbedText('{test_query}'), EmbedText(transcript)) LIMIT 05;
""").df()

,_row_id,start_time,src_url,transcript,video_id
0,11,219.66,https//www.youtube.com/watch?v=L_Guz73e6fw,"with many folks who now work at OpenAI, includ...",L_Guz73e6fw
1,230,4965.51,https//www.youtube.com/watch?v=L_Guz73e6fw,Yeah. What do you disagree on? Elon is obvio...,L_Guz73e6fw
2,235,5075.94,https//www.youtube.com/watch?v=L_Guz73e6fw,"in many instances, hes, like,\na very funny an...",L_Guz73e6fw
3,187,4019.94,https//www.youtube.com/watch?v=L_Guz73e6fw,and something he doesnt\nmake explicit but he ...,L_Guz73e6fw
4,111,2382.78,https//www.youtube.com/watch?v=L_Guz73e6fw,Im sure it has all\nsorts of subtle effects I...,L_Guz73e6fw


In [4]:
# create a simple flask endpoint to serve the five most similar games
from flask import Flask, request, jsonify
app = Flask(__name__)

# allow cross origin requests from any domain
from flask_cors import CORS
CORS(app)

@app.route('/get_similar_videos', methods=['GET'])
def get_similar_games():
    query = request.args.get('query')
    out = cursor.query(f"""
    SELECT * FROM videos ORDER BY Similarity(EmbedText('{query}'), EmbedText(transcript)) LIMIT 15;
    """).df()
    out_recs = out.to_dict(orient="records")
    # replace every url with https with https: (BC EVADB) doesn't let me insert w/ ?
    for rec in out_recs:
        rec["src_url"] = rec["src_url"].replace("https", "https:")
    return jsonify(out_recs)

app.run(host="localhost", port=5001)

# this can be accessed as follows: http://localhost:5001/get_similar_games?query=fast-paced%20action%20game

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5001
Press CTRL+C to quit
127.0.0.1 - - [22/Nov/2023 20:50:47] "GET /get_similar_videos?query=undefined HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 20:59:21] "GET /get_similar_videos?query=11%20bit%20studios HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 20:59:50] "GET /get_similar_videos?query=Reddit%20hostile%20takeover HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 21:00:54] "GET /get_similar_videos?query=to%20talk%20about%20AGI%20in%20the%20face%20of%20mockery.%20%20We%20dont%20get%20mocked%20as%20much%20now.%20%20We%20dont%20get%20mocked%20as%20much%20now.%20So,%20speaking%20about%20thestructure%20of%20the%20org.%20So,%20OpenAI%20stopped%20beingnonprofit%20or%20split%20up%20in%2020. HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2023 21:00:58] "GET /get_similar_videos?query=and%20badly%20mocked%20org%20for%20a%20long%20time.%20Like,%20when%20we%20started%20and%20we,%20like,%20announcedthe%20org%20at%20the%20end%20of%202015%20and%20said%20we%20were%20gonna%20work%20on%